In [ ]:
from datetime import datetime

import camelot
import pandas as pd

In [ ]:
# Configuration
PDF_FILE_PATH = './docs/statement-4.pdf'

# Column Headers from the Document
COLUMN_NAMES = [
    "Posted Date",
    "Value Date",
    "Description",
    "Debit",
    "Credit",
    "Balance"
]

def extract_and_clean_transactions(pdf_path: str) -> pd.DataFrame:
    """Extract, build, and clean transaction data from a PDF."""
    print("Starting extraction...")
    tables = camelot.read_pdf(pdf_path, pages='all', flavor='stream', row_tol=15)
    transactions_df = pd.concat([table.df for table in tables], ignore_index=True)
    transactions_df.dropna(axis=1, how='all', inplace=True)

    num_columns = min(len(COLUMN_NAMES), transactions_df.shape[1])
    transactions_df.columns = COLUMN_NAMES[:num_columns]

    print("Performing final data cleaning...")
    cleaned_df = transactions_df.copy()

    # Clean up newline characters and leading/trailing spaces
    for col in cleaned_df.columns:
        if cleaned_df[col].dtype == 'object':
            cleaned_df.loc[:, col] = (
                cleaned_df[col]
                .str.replace('\n', ' ', regex=False)
                .str.strip()
            )

    def clean_numeric_col(series: pd.Series) -> pd.Series:
        # Remove commas, letters, and any non-numeric characters except for '.' and '-'
        cleaned_series = series.astype(str).str.replace(',', '', regex=False)
        cleaned_series = cleaned_series.str.replace(r'[^\d.-]', '', regex=True)
        return pd.to_numeric(cleaned_series, errors='coerce')

    cleaned_df.loc[:, 'Debit'] = clean_numeric_col(cleaned_df['Debit'])
    cleaned_df.loc[:, 'Credit'] = clean_numeric_col(cleaned_df['Credit'])
    cleaned_df.loc[:, 'Balance'] = clean_numeric_col(cleaned_df['Balance'])

    cleaned_df.loc[:, 'Posted Date'] = pd.to_datetime(
        cleaned_df['Posted Date'],
        format="%d-%b-%y",
        errors='coerce'
    )
    cleaned_df.loc[:, 'Value Date'] = pd.to_datetime(
        cleaned_df['Value Date'],
        format="%d-%b-%y",
        errors='coerce'
    )

    cleaned_df = cleaned_df[cleaned_df['Value Date'].notna()]
    cleaned_df.reset_index(drop=True, inplace=True)

    print("Extraction and cleaning complete.")
    return cleaned_df

In [ ]:
if __name__ == '__main__':
    # --- Execute the script ---
    final_transactions_df = extract_and_clean_transactions(PDF_FILE_PATH)

    # --- Display Results ---
    print("\n--- Extracted Transactions ---")
    print(f"Total transactions extracted: {len(final_transactions_df)}")

    # Display the first 10 and last 10 transactions
    print("\nFirst 50 Transactions:")
    display(final_transactions_df.head(50))

    print("\nLast 50 Transactions:")
    display(final_transactions_df.tail(50))

    # --- Optional: Save to CSV ---
    try:
        output_csv_path = f'./output/extracted_transactions{datetime.now().strftime('%d%m%Y%H%M%S')}.csv'
        final_transactions_df.to_csv(output_csv_path, index=False)
        print(f"\nSuccessfully saved cleaned data to '{output_csv_path}'")
    except Exception as e:
        print(f"\nCould not save to CSV: {e}")